<a href="https://colab.research.google.com/github/gustavochavesferreira/tp1_algoritmos2/blob/main/LZW_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### Instalações provisórias para debug

In [156]:
!pip install graphviz
!apt-get install graphviz -y

from graphviz import Digraph

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


# Implementação da árvore de prefixos (Trie)

In [157]:
class CompactTrieNode:
  def __init__(self, binary_string: str = "", value: str = "", is_leaf: bool = False) -> None:
    self.children = [None, None]
    self.is_leaf = is_leaf
    self.binary_string = binary_string
    self.value = value
    self.unique_id = id(self)  # Temporário

class CompactTrie:
  def __init__(self) -> None:
    self.root = None

  def get_common_prefix_length(self, key1: str, key2: str) -> int:
    i = 0
    while i < min(len(key1), len(key2)) and key1[i] == key2[i]:
        i += 1
    return i

  def search(self, key: str) -> CompactTrieNode:
    current_node = self.root

    if(current_node == None):
      return None

    while True:
      if((current_node.binary_string == key) and (current_node.is_leaf == True)):
        return current_node

      common_prefix_length = self.get_common_prefix_length(current_node.binary_string, key)

      if(common_prefix_length == len(current_node.binary_string)):

        key = key[common_prefix_length:]

        if current_node.children[int(key[0])] == None:
          return None

        current_node = current_node.children[int(key[0])]
      else:
        return None

  def insert(self, key: str, value: str) -> None:
    # Caso em que não tínhamos raiz
    if(self.root == None):
      self.root = CompactTrieNode(key, value, True)
      return

    else:
      current_node = self.root

      while True:
        # Achei, posso parar
        if(current_node.binary_string == key):
          current_node.is_leaf = True
          return

        common_prefix_length = self.get_common_prefix_length(current_node.binary_string, key)

        # Sobrou parte da key ainda, com certeza
        if (common_prefix_length == len(current_node.binary_string)):
          key = key[common_prefix_length:]

          if current_node.children[int(key[0])] == None:
            current_node.children[int(key[0])] = CompactTrieNode(key, value, True)
            return

          current_node = current_node.children[int(key[0])]

        # Achei onde vou ter que criar novo nó para inserir
        else:
          key = key[common_prefix_length:]

          if(current_node.is_leaf == True):
            old_suffix_node = CompactTrieNode(current_node.binary_string[common_prefix_length:], current_node.value, True)
          else:
            old_suffix_node = CompactTrieNode(current_node.binary_string[common_prefix_length:], current_node.value)

          old_suffix_node.children[0] = current_node.children[0]
          old_suffix_node.children[1] = current_node.children[1]

          current_node.binary_string = current_node.binary_string[:common_prefix_length]

          if(len(key) > 0):
            key_suffix_node = CompactTrieNode(key, value, True)
            current_node.is_leaf = False
            current_node.children[int(key_suffix_node.binary_string[0])] = key_suffix_node
            current_node.children[int(old_suffix_node.binary_string[0])] = old_suffix_node
          else:
            current_node.value = value
            current_node.is_leaf = True
            current_node.children = [None, None]
            current_node.children[int(old_suffix_node.binary_string[0])] = old_suffix_node
          return

  def delete_key(self, key: str) -> None:
    if(self.search(key) == None):
      return

    current_node = self.root
    last_node = self.root

    if(current_node.binary_string == key):
      if(current_node.children[0] == None and current_node.children[1] == None):
        self.root = None
        return
      elif(current_node.children[0] != None and current_node.children[1] != None):
        current_node.is_leaf = False
        return
      else:
        valid_child = 0 if current_node.children[0] != None else 1
        current_node.children[valid_child].binary_string = current_node.binary_string + current_node.children[valid_child].binary_string
        self.root = current_node.children[valid_child]
        return

    while True:
      if((current_node.binary_string == key) and (current_node.is_leaf == True)):
        if((current_node.children[0] != None) and (current_node.children[1] != None)):
          current_node.is_leaf = False
        elif((current_node.children[0] == None) and (current_node.children[1] == None)):
          last_node.children[int(current_node.binary_string[0])] = None

          if(last_node.is_leaf == False):
            last_node_other_child = 0 if current_node.binary_string[0] == '1' else 1

            if(last_node.children[last_node_other_child] != None):
              last_node.binary_string += last_node.children[last_node_other_child].binary_string
              last_node.value = last_node.children[last_node_other_child].value
              if(last_node.children[last_node_other_child].is_leaf):
                last_node.is_leaf = True
              last_node.children = last_node.children[last_node_other_child].children
        else:
          valid_child = 0 if current_node.children[0] != None else 1
          current_node.children[valid_child].binary_string = current_node.binary_string + current_node.children[valid_child].binary_string
          last_node.children[int(current_node.children[valid_child].binary_string[0])] = current_node.children[valid_child]
        return

      common_prefix_length = self.get_common_prefix_length(current_node.binary_string, key)
      key = key[common_prefix_length:]
      last_node = current_node
      current_node = current_node.children[int(key[0])]

  # Temporário
  def visualize(self, filename="compact_trie"):
      dot = Digraph(comment="Compact Trie")

      def add_nodes_edges(node, parent_label=None):
          if node is None:
              return

          new_node_binary_string = node.binary_string
          new_node_binary_string = new_node_binary_string.replace("0", "a")
          new_node_binary_string = new_node_binary_string.replace("1", "b")

          node_label = f"{new_node_binary_string}_{node.unique_id}"

          display_label = f"{new_node_binary_string}"
          if(node.is_leaf):
            display_label  += f"\\nValue: {node.value}"

          dot.node(node_label, display_label, shape='circle', color='black', fontcolor='red' if node.is_leaf else 'blue')

          if parent_label:
              dot.edge(parent_label, node_label)
          if node.children[0] is not None:
              add_nodes_edges(node.children[0], node_label)
          if node.children[1] is not None:
              add_nodes_edges(node.children[1], node_label)

      if self.root is not None:
          add_nodes_edges(self.root)

      dot.render(filename, format="png", cleanup=True)
      print(f"Compact trie saved as {filename}.png")

# Implementando a versão final da compressão LZW, empregando árvores Trie como dicionários para consulta e inserção:

In [158]:
import os
from typing import Tuple
import struct

# Função para converter um conjunto de bytes em um '.txt'
def write_txt_file(decoded_bytes, output_file):
    decoded_string = ''.join([chr(byte) for byte in decoded_bytes])
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(decoded_string)

# Função para converter um conjunto de bytes em um '.bmp' ou '.tiff'
def write_image_file(decoded_bytes, output_file):
    with open(output_file, 'wb') as file:
        file.write(bytes(decoded_bytes))

# Função para remover zeros à esquerda em uma string binárias
def remove_leading_zeros(binary_str):
  return binary_str.lstrip('0') or '0'

class TrieLZW:
  # Realiza a compressão LZW do arquivo passado como parâmetro
  def compress(self, file_path: str="") -> Tuple[str, str]:
    # Utilizando a Trie Compacta Binária como dicionário no algoritmo
    dictionary = CompactTrie()

    # Inicializando todas as chaves de 00000000 até 11111111 no dicionário, com respectivos valores sendo a própria chave
    for num in range(256):
      byte_num = format(num, '08b')
      dictionary.insert(byte_num, byte_num)

    dict_size = 256 # O dicionário começa com todos os símbolos ASCII
    num_bits_values = 9 # Começaremos usando 9 bits para representar códigos a partir de agora
    next_dict_size_limit = 256 * 2 # Os 9 bits serão suficientes para representar códigos de 000000000 (0) até 111111111 (511)

    # Inicializando variáveis utilizadas pela compressão LZW
    string = ""
    compressed_data = []

    # Aplicando a compressão LZW, considerando cada byte do arquivo orignal como um símbolo de entrada
    try:
      with open(file_path, "rb") as file:
          while (byte := file.read(1)):
              symbol = bin(int.from_bytes(byte, "big"))[2:].zfill(8)

              string_plus_symbol = string + symbol
              if dictionary.search(string_plus_symbol) != None:
                  string = string_plus_symbol
              else:
                  # Caso em que o dicionário tem tamanho dinâmico
                  if(dict_size == next_dict_size_limit):
                      num_bits_values += 1
                      next_dict_size_limit *= 2

                  current_string_formatted_binary_value = dictionary.search(string).value.zfill(num_bits_values)
                  compressed_data.append(current_string_formatted_binary_value)
                  new_key_value = bin(dict_size)[2:].zfill(num_bits_values)
                  dictionary.insert(string_plus_symbol, new_key_value)
                  dict_size += 1
                  string = symbol

          if(dictionary.search(string) != None):
            current_string_formatted_binary_value = dictionary.search(string).value.zfill(num_bits_values)
            compressed_data.append(current_string_formatted_binary_value)

    # Tratando erros que podem ocorrer na abertura de um arquivo
    except FileNotFoundError as e:
      print(f"Arquivo não encontrado -> {e}")
    except IOError as e:
      print(f"Erro de I/O -> {e}")
    except Exception as e:
      print(f"Um erro ocorreu: {e}")

    # Salvando o nome do arquivo original e sua extensão para que o arquivo de compressão possa ser salvo
    file_name, file_extension = os.path.splitext(file_path)

    with open(f"compressed_{file_name}.bin", "wb") as file:
      # Convertendo os códigos para uma única string
      final_concat_string_data = ''.join(compressed_data)

      # Verificando se algum padding deverá ser adicionado para que tenhamos um número inteiro de bytes no arquivo comprimido
      padding_size = (8 - len(final_concat_string_data) % 8) % 8
      final_concat_string_data = '0' * padding_size + final_concat_string_data

      # Adicionando uma flag, no arquivo comprimido, para representar o tamanho do padding que foi adicionado
      file.write(bytes([padding_size]))

      # Convertendo a string de dados comprimidos para bytes e escrevendo no arquivo comprimido final
      bits = []
      for char in final_concat_string_data:
          bits.append(1 if char == '1' else 0)
      byte = 0
      bit_count = 0
      for bit in bits:
          byte = (byte << 1) | bit
          bit_count += 1
          if bit_count == 8:
              file.write(bytes([byte]))
              byte = 0
              bit_count = 0

      return [file_name, file_extension]

  # Realiza a descompressão LZW do arquivo passado como parâmetro
  def decompress(self, file_path: str="", original_file_name: str="", original_file_extension: str="") -> None:
    # Utilizando a Trie Compacta Binária como dicionário no algoritmo
    dictionary = CompactTrie()

    # Inicializando todas as chaves de 0, 1, 10, 11, 100, ... até 11111111 no dicionário (sem padding), com respectivos valores sendo a própria chave (com padding)
    for num in range(256):
      numeric_key = bin(num)[2:]
      numeric_key_8bit_value = format(num, '08b')
      dictionary.insert(numeric_key, numeric_key_8bit_value)

    dict_size = 256 # O dicionário começa com todos os número de 0 até 255, cada um representando um símbolo ASCII
    decompression_size = 9 # Começaremos puxando 9 bits do arquivo comprimido por vez
    next_size_limit = 256 * 2 # Os 9 bits serão suficientes até que o tamanho do dicionário atinja 512 elementos

    # Abrindo o arquivo já comprimido e convertendo seus bytes em uma única string
    compressed_data = ""
    try:
        with open(file_path, "rb") as file:
            file_data = file.read()
            compressed_data = ''.join(format(byte, '08b') for byte in file_data)
    # Tratando erros que podem ocorrer na abertura de um arquivo
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except IOError as e:
        print(f"Error reading the file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    # Retirando os bits de padding do arquivo comprimido
    padding_size = int(compressed_data[:8], 2)
    compressed_data = compressed_data[(8 + padding_size):]

    # Inicializando as variáveis que serão empregadas posteriormente
    string = compressed_data[:decompression_size]
    string = string[-8:]
    compressed_data = compressed_data[decompression_size:]
    decompressed_data = [string]

    # Aplicando a descompressão LZW
    while(len(compressed_data) > 0):
        # Caso em que o dicionário tem tamanho dinâmico
        if(dict_size == next_size_limit):
                decompression_size += 1
                next_size_limit *= 2

        k = compressed_data[:decompression_size]
        compressed_data = compressed_data[decompression_size:]

        k = remove_leading_zeros(k)

        if dictionary.search(k) != None:
            entry = dictionary.search(k).value
        else:
            new_value_entry_concat = string[:8]
            entry = string + new_value_entry_concat

        decompressed_data.append(entry)
        new_key = bin(dict_size)[2:] if dict_size != 0 else '0'
        dictionary.insert(new_key, string + entry[:8])
        dict_size += 1
        string = entry

    decompressed_concat_string = ''.join(decompressed_data)

    # Checando algum possível erro que possa ter ocorrido na descompressão (o tamanho do arquivo final deve ser um múltiplo de 8)
    if len(decompressed_concat_string) % 8 != 0:
      raise ValueError("O arquivo descomprimido não tem um número inteiro de bytes!")

    # Transformando a string binárias em bytes individuais
    byte_chunks = [decompressed_concat_string[i:i+8] for i in range(0, len(decompressed_concat_string), 8)]
    bytes_ = [int(chunk, 2) for chunk in byte_chunks]

    # Gerando o arquivo inicial
    switch = {
        '.txt': write_txt_file,
        '.bmp': write_image_file,
        '.tiff': write_image_file # Não sei se está funcionando
        }
    handler = switch.get(file_extension)
    if handler:
        handler(bytes_, f"decompressed_{original_file_name}{original_file_extension}")
    else:
        print("Unsupported file type.")

# Fazendo a compressão e a descompressão de arquivos usando a classe TrieLZW:

Função temporária para verificar se dois arquivos são iguais:

In [159]:
def are_files_identical(file1, file2):
    """
    Compare two files byte by byte to check if they are identical.
    """
    try:
        with open(file1, "rb") as f1, open(file2, "rb") as f2:
            while True:
                chunk1 = f1.read(4096)  # Read in chunks of 4 KB
                chunk2 = f2.read(4096)
                if chunk1 != chunk2:  # Compare the current chunks
                    return False
                if not chunk1:  # End of file reached
                    return True
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return False

### Exemplo 1 (arquivo de texto)

Gerando artificialmente um texto para teste:

In [160]:
import random

# Generate the random text
input_data = ''.join(random.choices("abcd", k=100))

# Specify the file path
file_path = "test_text.txt"

# Save the text to a .txt file
with open(file_path, "w") as file:
    file.write(input_data)

Aplicando a compressão e a descompressão em sequência:

In [161]:
teste_trie_lzw = TrieLZW()

file_to_be_compressed = "test_text.txt"

[file_name, file_extension] = teste_trie_lzw.compress(file_to_be_compressed)

compressed_file = f"compressed_{file_name}.bin"

teste_trie_lzw.decompress(compressed_file, file_name, file_extension)

Comparando o tamanho dos arquivos iniciais e finais:

In [162]:
original_file_size = os.path.getsize(file_to_be_compressed)
compressed_file_size = os.path.getsize(compressed_file)

print(f"Tamanho do arquivo original: {original_file_size} bytes.")
print(f"Tamanho do arquivo comprimido: {compressed_file_size} bytes.")

Tamanho do arquivo original: 100 bytes.
Tamanho do arquivo comprimido: 60 bytes.


Verificando se os arquivos são equivalentes:

In [163]:
txt_file1 = file_to_be_compressed
txt_file2 = f"decompressed_{file_to_be_compressed}"
print(txt_file1)
print(txt_file2)
print(f"Os arquivos .txt são iguais? {are_files_identical(txt_file1, txt_file2)}")

test_text.txt
decompressed_test_text.txt
Os arquivos .txt são iguais? True


### Exemplo 2 (arquivo de imagem)

Gerando artificialmente uma imagem para teste:

In [164]:
from PIL import Image

# Create a new monochrome (mode '1') image with 10x10 pixels
width, height = 100, 100
image = Image.new("1", (width, height))  # Mode "1" means 1-bit pixels (monochrome)

# Set some pixels to black (1 = white, 0 = black)
pixels = image.load()
for x in range(width):
    for y in range(height):
        if (x + y) % 2 == 0:  # Example pattern: checkerboard
            pixels[x, y] = 0  # Black pixel
        else:
            pixels[x, y] = 1  # White pixel

# Save the image as a BMP file
image.save("monochrome_test_image.bmp")

Aplicando a compressão e a descompressão em sequência:

In [165]:
teste_trie_lzw = TrieLZW()

file_to_be_compressed = "monochrome_test_image.bmp"

[file_name, file_extension] = teste_trie_lzw.compress(file_to_be_compressed)

compressed_file = f"compressed_{file_name}.bin"

teste_trie_lzw.decompress(compressed_file, file_name, file_extension)

Comparando o tamanho dos arquivos iniciais e finais:

In [166]:
original_file_size = os.path.getsize(file_to_be_compressed)
compressed_file_size = os.path.getsize(compressed_file)

print(f"Tamanho do arquivo original: {original_file_size} bytes.")
print(f"Tamanho do arquivo comprimido: {compressed_file_size} bytes.")

Tamanho do arquivo original: 1662 bytes.
Tamanho do arquivo comprimido: 286 bytes.


Verificando se os arquivos são equivalentes:

In [167]:
bmp_file1 = file_to_be_compressed
bmp_file2 = f"decompressed_{file_to_be_compressed}"

print(f"Os arquivos .bmp são iguais? {are_files_identical(bmp_file1, bmp_file2)}")

Os arquivos .bmp são iguais? True


# Seção provisória para debug

In [168]:
import struct

def int_to_8bit_binary(number):
  return format(number, '08b')

def remove_leading_zeros(binary_str):
  return binary_str.lstrip('0') or '0'

class LZW:
    def __init__(self):
        self.max_table_size = 4096

    def compress(self, input_data):
        dictionary = {chr(i): i for i in range(256)}
        string = ""
        compressed_data = []

        for symbol in input_data:
            string_plus_symbol = string + symbol
            if string_plus_symbol in dictionary:
                string = string_plus_symbol
            else:
                compressed_data.append(dictionary[string])
                if len(dictionary) < self.max_table_size:
                    dictionary[string_plus_symbol] = len(dictionary)
                string = symbol

        if string in dictionary:
            compressed_data.append(dictionary[string])

        return compressed_data

    def decompress(self, compressed_data):
        dictionary = {chr(i): i for i in range(256)}
        string = chr(compressed_data.pop(0))
        decompressed_data = [string]

        for k in compressed_data:
            print(k)
            if k in dictionary:
                entry = dictionary[k]
            elif k == len(dictionary):
                entry = string + string[0]
            else:
                raise ValueError("Erro na compressão k: %s" % k)

            decompressed_data.append(entry)

            if len(dictionary) < self.max_table_size:
                dictionary[len(dictionary)] = string + entry[0]

            string = entry

        return ''.join(decompressed_data)

In [169]:
'''
import os
import random

if __name__ == "__main__":
    lzw = LZW()

    input_data = ''.join(random.choices("ab", k=1000))

    input_data = 'geekific-geekific'

    with open("input.txt", 'w') as file:
      file.write(input_data)

    with open("input.txt", "r") as file:
      input_data = file.read()

    with open('input.txt', 'rb') as file:
      input_data_2 = file.read()
      input_data_2 = list(input_data_2)

    compressed_data = lzw.compress(input_data)
    #compressed_data_trie = lzw.compress_trie(input_data_2)
    #integer_compressed_data_trie = [int(binary, 2) for binary in compressed_data_trie]

    #print("A compressão ficou igual!") if compressed_data == integer_compressed_data_trie else print("A compressão ficou diferente!")
    print("Arquivo comprimido (dict):", compressed_data)
    #print("Arquivo comprimido (trie):", integer_compressed_data_trie)
    #print("Arquivo comprimido original (trie):", compressed_data_trie)

    with open("text_compressed_result.txt", 'w') as file:
      final_compressed_string = ''.join(compressed_data_trie)
      file.write(final_compressed_string)

    with open("binary_compressed_result.bin", 'wb') as file:
      final_compressed_string = ''.join(compressed_data_trie)

      padding_size = (8 - len(final_compressed_string) % 8) % 8
      final_compressed_string = '0' * padding_size + final_compressed_string

      file.write(bytes([padding_size]))

      bits = []
      for char in final_compressed_string:
          bits.append(1 if char == '1' else 0)

      byte = 0
      bit_count = 0
      for bit in bits:
          byte = (byte << 1) | bit
          bit_count += 1

          if bit_count == 8:
              file.write(bytes([byte]))
              byte = 0
              bit_count = 0

    size_file1 = os.path.getsize("input.txt")
    size_file2 = os.path.getsize("binary_compressed_result.bin")

    with open("binary_compressed_result.bin", 'rb') as file:
      padding_size = ord(file.read(1))

      data = file.read()

      binary_string = ''.join(format(byte, '08b') for byte in data)

      binary_string = binary_string[padding_size:]

    result = lzw.decompress_trie(binary_string)

    print(" O resultado antes da compressão é: " + input_data)
    print("O resultado após a descompressão é: " + result)
    print(f"Tamanho do arquivo original: {size_file1} bytes")
    print(f"Tamanho do arquivo comprimido: {size_file2} bytes")
    print("O LZW foi bem-sucedido!") if result == input_data else print("O LZW não foi bem-sucedido!")

    with open("compressed_binary.txt", "w") as file:
        binary_data = ' '.join(format(data, '012b') for data in compressed_data)
        file.write(binary_data)

    decompressed_data = lzw.decompress(compressed_data)
    print("Arquivo descomprimido:", decompressed_data)

    with open("decompressed.txt", "w") as file:
        file.write(decompressed_data)
    '''

'\nimport os\nimport random\n\nif __name__ == "__main__":\n    lzw = LZW()\n\n    input_data = \'\'.join(random.choices("ab", k=1000))\n\n    input_data = \'geekific-geekific\'\n\n    with open("input.txt", \'w\') as file:\n      file.write(input_data)\n\n    with open("input.txt", "r") as file:\n      input_data = file.read()\n\n    with open(\'input.txt\', \'rb\') as file:\n      input_data_2 = file.read()\n      input_data_2 = list(input_data_2)\n\n    compressed_data = lzw.compress(input_data)\n    #compressed_data_trie = lzw.compress_trie(input_data_2)\n    #integer_compressed_data_trie = [int(binary, 2) for binary in compressed_data_trie]\n\n    #print("A compressão ficou igual!") if compressed_data == integer_compressed_data_trie else print("A compressão ficou diferente!")\n    print("Arquivo comprimido (dict):", compressed_data)\n    #print("Arquivo comprimido (trie):", integer_compressed_data_trie)\n    #print("Arquivo comprimido original (trie):", compressed_data_trie)\n\n   

In [170]:
'''
import os
from IPython.display import Image
from random import choice
from IPython.display import display, clear_output
import ipywidgets as widgets

trie = CompactTrie()

k = bin(0)[2:]
v = format(0, '08b')

trie.insert(k, v)

file_path = "interactive_trie.png"
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted.")
else:
    print(f"{file_path} does not exist.")
trie.visualize("interactive_trie")
display(Image(filename="interactive_trie.png"))
'''

'\nimport os\nfrom IPython.display import Image\nfrom random import choice\nfrom IPython.display import display, clear_output\nimport ipywidgets as widgets\n\ntrie = CompactTrie()\n\nk = bin(0)[2:]\nv = format(0, \'08b\')\n\ntrie.insert(k, v)\n\nfile_path = "interactive_trie.png"\nif os.path.exists(file_path):\n    os.remove(file_path)\n    print(f"{file_path} has been deleted.")\nelse:\n    print(f"{file_path} does not exist.")\ntrie.visualize("interactive_trie")\ndisplay(Image(filename="interactive_trie.png"))\n'

In [171]:
import os
from IPython.display import Image
from random import choice
from IPython.display import display, clear_output
import ipywidgets as widgets

trie = CompactTrie()
global_counter = 0
delete_input = widgets.Text(description="Delete:")
delete_button = widgets.Button(description="Delete String")

def reset_trie(_):
    global trie, global_counter
    trie = CompactTrie()
    global_counter = 0

    clear_output(wait=True)

    auto_button = widgets.Button(description="Auto")
    auto_button.on_click(random_insert)
    display(auto_button)

def random_insert(_):
    global global_counter
    global_counter += 1
    clear_output(wait=True)

    iterate_button = widgets.Button(description="Next Iteration")
    iterate_button.on_click(random_insert)
    display(iterate_button)

    delete_button.on_click(delete_string)
    display(delete_input)
    display(delete_button)

    reset_button = widgets.Button(description="Reset Trie")
    reset_button.on_click(reset_trie)
    display(reset_button)

    new_node_string = ''.join(choice(['a', 'b']) for _ in range(6))
    print(f"Generated string: {new_node_string}")

    new_binary_string = ''.join('0' if char == 'a' else '1' for char in new_node_string)

    trie.insert(new_binary_string, global_counter)

    file_path = "interactive_trie.png"

    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"{file_path} has been deleted.")
    else:
        print(f"{file_path} does not exist.")

    trie.visualize("interactive_trie")

    display(Image(filename="interactive_trie.png"))

def delete_string(_):
    global delete_input
    binary_string = delete_input.value.strip()
    new_binary_string = ''.join('0' if char == 'a' else '1' for char in binary_string)
    trie.delete_key(new_binary_string)

    clear_output(wait=True)

    iterate_button = widgets.Button(description="Next Iteration")
    iterate_button.on_click(random_insert)
    display(iterate_button)

    delete_button.on_click(delete_string)
    display(delete_input)
    display(delete_button)

    reset_button = widgets.Button(description="Reset Trie")
    reset_button.on_click(reset_trie)
    display(reset_button)

    file_path = "interactive_trie.png"

    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"{file_path} has been deleted.")
    else:
        print(f"{file_path} does not exist.")

    trie.visualize("interactive_trie")

    display(Image(filename="interactive_trie.png"))

auto_button = widgets.Button(description="Auto")
auto_button.on_click(random_insert)
display(auto_button)

Button(description='Auto', style=ButtonStyle())